In [1]:
import win32com.client
import os
from datetime import datetime
from collections import OrderedDict
import time
import fitz
import pytz    
import string


def criar_arquivo_txt(linhas):
    dic_atual = os.getcwd()
    diretorio = dic_atual+r'\favorecido.txt'
    print(diretorio)
    with open(diretorio, 'w') as arquivo:
        for linha in linhas:
            arquivo.write(linha + '\n')


total_lote = 0
total_pagamento = 0
Excel = win32com.client.Dispatch("Excel.Application")
Workbook = Excel.ActiveWorkbook

fuso_horario_brasil = pytz.timezone('America/Sao_Paulo')

data_hora_brasil = datetime.now(fuso_horario_brasil)

data_hora_formatada = data_hora_brasil.strftime("%d-%m-%Y %H:%M:%S")
data = data_hora_formatada.split(' ')[0]
hora = data_hora_formatada.split(' ')[1]

data = data.replace('-','')
hora = hora.replace(':','')

letras = list(string.ascii_uppercase)[:26]

for letra in letras:
    if str(Workbook.Worksheets[3].Range(f"{letra}1").Value)=="N° DE INSCRIÇÃO":
        total_linhas=2
        while True:
            if Workbook.Worksheets[3].Range(f"{letra}{total_linhas}").Value==None:
                print(total_linhas)
                break
            total_linhas=total_linhas+1

header_arq = []
x=0
linha_header_arq=''
for letra in letras:
    header_arq.append(str(Workbook.Worksheets[0].Range(f"{letra}2").Value))
    coluna = header_arq[x].split('.')[0]

    largura1 = str(Workbook.Worksheets[6].Range("B"+str(x+2)).Value)
    largura2 = str(Workbook.Worksheets[6].Range("B"+str(x+3)).Value)
    testehoras = largura1.split('-')[0]
    if largura1!="None" and largura2!="None":
        esp = largura1.split('-')[2]
        largura1 = largura1.split('-')[1]
        largura2 = largura2.split('-')[0]

    if coluna=="None":
        coluna=' '*int(esp)
    if str(Workbook.Worksheets[0].Range(f"{letra}1").Value)=="DATA":
        coluna=data
    if str(Workbook.Worksheets[0].Range(f"{letra}1").Value)=="HORA":
        coluna=hora

    numero_de_caracteres = len(coluna)

    if numero_de_caracteres!= esp:
        sub = int(esp)-int(numero_de_caracteres)
        coluna = coluna+f"{' '*sub}"

    if largura1!="None" and largura2!="None":
        largura = int(largura2)-int(largura1)
        linha_header_arq = linha_header_arq+f"{coluna}{' '*(largura-1)}"
    x=x+1
linha_header_arq = linha_header_arq+f"{' '*(240-len(linha_header_arq))}"

header_lote = []
linha_header_lote=''
x=0
for letra in letras:
    header_lote.append(str(Workbook.Worksheets[1].Range(f"{letra}2").Value))
    coluna = header_lote[x].split('.')[0]

    largura1 = str(Workbook.Worksheets[6].Range("C"+str(x+2)).Value)
    largura2 = str(Workbook.Worksheets[6].Range("C"+str(x+3)).Value)
    if largura1!="None" and largura2!="None":
        esp = largura1.split('-')[2]
        largura1 = largura1.split('-')[1]
        largura2 = largura2.split('-')[0]

    if coluna=="None":
        coluna=' '*int(esp)

    numero_de_caracteres = len(coluna)

    if numero_de_caracteres!= esp:
        sub = int(esp)-int(numero_de_caracteres)
        coluna = coluna+f"{' '*sub}"

    if largura1!="None" and largura2!="None":
        largura = int(largura2)-int(largura1)
        linha_header_lote = linha_header_lote+f"{coluna}{' '*(largura-1)}"
    x=x+1
linha_header_lote = linha_header_lote+f"{' '*(240-len(linha_header_lote))}"

lote_seg_b = []
lote_seg_a = []

for z in range(2, total_linhas):
    lote_a = []
    linha_lote_a=''
    lote_b = []
    linha_lote_b=''
    x=0
    for letra in letras:
        lote_a.append(str(Workbook.Worksheets[2].Range(f"{letra}{z}").Value))
        coluna = lote_a[x].split('.')[0]

        if str(Workbook.Worksheets[2].Range(f"{letra}1").Value)=="PAGAMENTO":
            valor_pagamento = round(float(lote_a[x]), 2)
            
            decimal = str(valor_pagamento).split(".")[1]
            total_pagamento = total_pagamento + valor_pagamento
            if len(decimal)!=2:
                decimal = 2-len(decimal)
                valor_pagamento = str(valor_pagamento)+f"{'0'*decimal}"
                
            coluna = str(valor_pagamento)
            coluna = coluna.split('.')[0]+coluna.split('.')[1]
            lin = len(coluna)
            sub = 15-int(lin)
            coluna = f"{'0'*sub}"+coluna
        if str(Workbook.Worksheets[2].Range(f"{letra}1").Value)=="Conta Corrente":
            coluna = coluna.replace("-","")
            for let in letras:
                if str(Workbook.Worksheets[2].Range(f"{let}1").Value)=="Digito Verificador":
                    if Workbook.Worksheets[2].Range(f"{let}{z}").Value==None:
                        digito_verificador = coluna[-1]
                        coluna = coluna[:-1]
                    else:
                        digito_verificador=str(Workbook.Worksheets[2].Range(f"{let}{z}").Value)
                        digito_verificador = digito_verificador.split('.')[0]
                    break
            lin = len(coluna)
            sub = 12-int(lin)
            coluna = f"{'0'*sub}"+coluna
        if str(Workbook.Worksheets[2].Range(f"{letra}1").Value)=="Digito Verificador":
            coluna = digito_verificador
        if str(Workbook.Worksheets[2].Range(f"{letra}1").Value)=="CONTA":
            coluna = coluna.replace("-","")
            coluna = coluna[:-1]+' '+coluna[-1]
            lin = len(coluna)
            sub = 13-int(lin)
            coluna = f"{'0'*sub}"+coluna
        if str(Workbook.Worksheets[2].Range(f"{letra}1").Value)=="AGÊNCIA":
            lin = len(coluna)
            sub = 5-int(lin)
            coluna = f"{'0'*sub}"+coluna
        if str(Workbook.Worksheets[2].Range(f"{letra}1").Value)=="Ag. Mantenedora":
            lin = len(coluna)
            sub = 5-int(lin)
            coluna = f"{'0'*sub}"+coluna
        if str(Workbook.Worksheets[2].Range(f"{letra}1").Value)=="BANCO FAVORECIDO":
            lin = len(coluna)
            sub = 3-int(lin)
            coluna = f"{'0'*sub}"+coluna
        if str(Workbook.Worksheets[2].Range(f"{letra}1").Value)=="N° DE INSCRIÇÃO":
            if len(lote_a[x])>13:
                coluna = lote_a[x].replace(".","")
                coluna = coluna.replace("/","")
                coluna = coluna.replace("-","")
            lin = len(coluna)
            sub = 14-int(lin)
            coluna = f"{'0'*sub}"+coluna
        if str(Workbook.Worksheets[2].Range(f"{letra}1").Value)=="TIPO DE MOVIMENTO ITAU":
            lin = len(coluna)
            sub = 3-int(lin)
            coluna = f"{'0'*sub}"+coluna
        largura1 = str(Workbook.Worksheets[6].Range("D"+str(x+2)).Value)
        largura2 = str(Workbook.Worksheets[6].Range("D"+str(x+3)).Value)
        testehoras = largura1.split('-')[0]
        if largura1!="None":
            esp = largura1.split('-')[2]
            largura1 = largura1.split('-')[1]
            largura2 = largura2.split('-')[0]

        if coluna=="None":
            coluna=' '*int(esp)

        if str(Workbook.Worksheets[2].Range(f"{letra}1").Value)=="DATA":
            coluna=data

        numero_de_caracteres = len(coluna)

        if numero_de_caracteres!= esp:
            sub = int(esp)-int(numero_de_caracteres)
            coluna = coluna+f"{' '*sub}"

        if largura1!="None" and largura2!="None":
            largura = int(largura2)-int(largura1)
            linha_lote_a = linha_lote_a+f"{coluna}{' '*(largura-1)}"
        elif largura1!="None":
            linha_lote_a = linha_lote_a+f"{coluna}"

        lote_b.append(str(Workbook.Worksheets[3].Range(f"{letra}{z}").Value))
        coluna = lote_b[x].split('.')[0]

        largura1 = str(Workbook.Worksheets[6].Range("E"+str(x+2)).Value)
        largura2 = str(Workbook.Worksheets[6].Range("E"+str(x+3)).Value)
        testehoras = largura1.split('-')[0]
        if largura1!="None":
            esp = largura1.split('-')[2]
            largura1 = largura1.split('-')[1]
            largura2 = largura2.split('-')[0]

        if coluna=="None":
            coluna=' '*int(esp)

        if str(Workbook.Worksheets[3].Range(f"{letra}1").Value)=="DATA":
            coluna=data 
        if str(Workbook.Worksheets[3].Range(f"{letra}1").Value)=="N° DE INSCRIÇÃO":
            if len(lote_b[x])>13:
                coluna = lote_b[x].replace(".","")
                coluna = coluna.replace("/","")
                coluna = coluna.replace("-","")
            lin = len(coluna)
            sub = 14-int(lin)
            coluna = f"{'0'*sub}"+coluna
            
        numero_de_caracteres = len(coluna)
        if numero_de_caracteres!= esp:
            sub = int(esp)-int(numero_de_caracteres)
            coluna = coluna+f"{' '*sub}"

        if largura1!="None" and largura2!="None":
            largura = int(largura2)-int(largura1)
            linha_lote_b = linha_lote_b+f"{coluna}{' '*(largura-1)}"
        elif largura1!="None":
            linha_lote_b = linha_lote_b+f"{coluna}"
            
        x=x+1
    linha_lote_b = linha_lote_b+f"{' '*(240-len(linha_lote_b))}"
    linha_lote_a = linha_lote_a+f"{' '*(240-len(linha_lote_a))}"
    total_lote=total_lote+2
    linha_lote_a = [linha_lote_a, linha_lote_b]
    lote_seg_a = lote_seg_a+linha_lote_a

trailer_lote = []
linha_trailer_lote=''
x=0
for letra in letras:
    trailer_lote.append(str(Workbook.Worksheets[4].Range(f"{letra}2").Value))
    coluna = trailer_lote[x].split('.')[0]
    if letra=="F":
        valor_pagamento = round(float(total_pagamento), 2)
        coluna = str(valor_pagamento)
        coluna = coluna.split('.')[0]+coluna.split('.')[1]
        lin = len(coluna)
        sub = 18-int(lin)
        coluna = f"{'0'*sub}"+coluna

    largura1 = str(Workbook.Worksheets[6].Range("F"+str(x+2)).Value)
    largura2 = str(Workbook.Worksheets[6].Range("F"+str(x+3)).Value)
    if largura1!="None" and largura2!="None":
        esp = largura1.split('-')[2]
        largura1 = largura1.split('-')[1]
        largura2 = largura2.split('-')[0]

    if letra=="E":
        coluna=str(total_lote+2)
        lin = len(coluna)
        sub = 6-int(lin)
        coluna = f"{'0'*sub}"+coluna

    if coluna=="None":
        coluna=' '*int(esp)

    numero_de_caracteres = len(coluna)

    if numero_de_caracteres!= esp:
        sub = int(esp)-int(numero_de_caracteres)
        coluna = coluna+f"{' '*sub}"

    if largura1!="None" and largura2!="None":
        largura = int(largura2)-int(largura1)
        linha_trailer_lote = linha_trailer_lote+f"{coluna}{' '*(largura-1)}"
    x=x+1

linha_trailer_lote = linha_trailer_lote+f"{' '*(240-len(linha_trailer_lote))}"
trailer_arq = []
linha_trailer_arq=''
x=0

for letra in letras:
    trailer_arq.append(str(Workbook.Worksheets[5].Range(f"{letra}2").Value))
    coluna = trailer_arq[x].split('.')[0]

    largura1 = str(Workbook.Worksheets[6].Range("G"+str(x+2)).Value)
    largura2 = str(Workbook.Worksheets[6].Range("G"+str(x+3)).Value)

    if largura1!="None" and largura2!="None":
        esp = largura1.split('-')[2]
        largura1 = largura1.split('-')[1]
        largura2 = largura2.split('-')[0]
    if letra=="F":
        coluna=str(total_lote+4)
        lin = len(coluna)
        sub = 6-int(lin)
        coluna = f"{'0'*sub}"+coluna

    if coluna=="None":
        coluna=' '*int(esp)

    numero_de_caracteres = len(coluna)

    if numero_de_caracteres!= esp:
        sub = int(esp)-int(numero_de_caracteres)
        coluna = coluna+f"{' '*sub}"

    if largura1!="None" and largura2!="None":
        largura = int(largura2)-int(largura1)
        linha_trailer_arq = linha_trailer_arq+f"{coluna}{' '*(largura-1)}"
    x=x+1

linha_trailer_arq = linha_trailer_arq+f"{' '*(240-len(linha_trailer_arq))}"

print("Concluido")
criar_arquivo_txt([linha_header_arq, linha_header_lote]+lote_seg_a+[linha_trailer_lote, linha_trailer_arq])

114
Concluido
C:\Users\leandro.arruda\Desktop\Backup\AutomaçãoFinanceiro\favorecido.txt
